<img src=../images/gdd-logo.png width=300px align=right>

# Dimensionality Reduction

Another form of unsupervised learning, besides clustering, is what we call _dimensionality reduction_. The dimensionality of data can be reduced through feature selection (e.g. missing value ratio, low variance or high correlation filter) or based on a supervised learning method such as Lasso with `SelectFromModel` in sklearn. 

However, there are also _unsupervised_ approaches to dimensionality reduction. Dimensionality reduction, and therefore PCA, has two applications: 
* Data compression
* Visualisation 

**Data compression** can be especially useful in the case that you have many, many features as it allows you to compress e.g. 1000 features into simply 100 features. This obviously cannot happen without some form of data loss, but the magic of dimensionality reduction is that it tries to limit the information loss. For example, say you have two features: height in cm and height in inches where in both cases the values have been rounded off. These are highly redundant as they are highly correlated and therefore quite easy to reduce to one single feature. 

**Visualisation** is also often a goal of dimensionality reduction. Visual inspection of your data can be hugely beneficial, but anything bigger than 3D data becomes problematic to visualise. In such case, we might reduce our dimensionality to two or three compontents in order to visualise our data.

In this notebook, we will take a look into different dimensionality reduction techniques that allow us to visualize the data.

# Dimensionality reduction for visualisation

- [PCA]()
- [t-SNE]()
- [UMAP]()

In [ ]:
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_openml

## The Data

In order to demonstrate dimensionality reduction techniques, we require some high-dimensional data. In this case, images. 

In [ ]:
# Load data from https://www.openml.org/d/554
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

In [ ]:
y = y.astype('int')

Let us load in the MNIST dataset and examine their shapes. The data consists of images of 28x28 pixels, where each of the 784 pixels in total is a feature. 

In [ ]:
print(X.shape)
print(y.shape)

Unsupervised learning approaches, especially dimensionality reduction for visualisation, do not require a test set. So we do not need to perform a train/test split.

In [ ]:
X = X / 255.0
print(X.shape)

Some preprocessing: normalizing the X values to a 0 - 1 range. 

### Visualize the data

In [ ]:
plt.figure(figsize=(15, 20))

examples = 5
for i in range(10): 
    indices = np.where(y == i)[0]
    for j in range(examples):
        img = X[indices[j]].reshape((28, 28))
        plt.subplot(10, examples, (i * examples) + j + 1)
        plt.imshow(img, cmap=plt.get_cmap('gray'))

## PCA for Visualization

Principal Component Analysis can be used to reduce the number of dimensions. PCA is a **projection** based unsupervised learning technique, which means this technique deals with projecting every data point which is in high dimension, onto a subspace suitable lower-dimensional space in a way which approximately preserves the distances between the points.

In [ ]:
from sklearn.decomposition import PCA

time_start = time.time()

pca = PCA()
pca_data = pca.fit_transform(X)

time_pca = time.time() - time_start
print(f'PCA done in {time_pca} seconds!')

In [ ]:
times = {}
times['PCA'] = time_pca

In [ ]:
fig = plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In order to maintain all or nearly all of the information contained in all the pixels, we would need approximately 200 components. However, that would be still be far too many dimensions to visualize. Let's see how much we can do with just two.

In [ ]:
print(f'Explained variance with 3 components: {np.sum(pca.explained_variance_ratio_[:3])}')

That's not necessarily a lot - let's see what it looks like in 2D though. 

In [ ]:
df = pd.DataFrame({
    'label': y,
    'pc1': pca_data[:, 0],
    'pc2': pca_data[:, 1],
    'pc3': pca_data[:, 2],
})

df.head()

In [ ]:
def plot_2d(df, x_axis, y_axis):
    plt.figure(figsize=(15, 10))
    sns.scatterplot(
    x = df[x_axis],
    y = df[y_axis],
    hue = 'label',
    data = df,
    palette=sns.color_palette("hls", 10),
    legend="full"
)
    

In [ ]:
plot_2d(df, 'pc1', 'pc2')

From the plot we can see that two components definitely hold some information, especially for specific digits, but not clearly enough to set all of them apart. 

Let's see if we can improve this with a 3D visualisation. 

In [ ]:
print(f'Explained variance with 3 components: {np.sum(pca.explained_variance_ratio_[:3])}')

We maintain quite a bit more information with three components than with two. 

In [ ]:
fig = plt.figure(figsize=(15, 20))
ax = fig.add_subplot(projection='3d')

ax.scatter(
    xs = df['pc1'],
    ys = df['pc2'],
    zs = df['pc3'],
    c = df['label'],
    cmap = 'tab10'
)

ax.set_xlabel('pc1')
ax.set_ylabel('pc2')
ax.set_zlabel('pc3')

Although we can create a 3D plot with matplotlib, it is not interactive and does not really allow us to fully explore the data. Instead, we will use plotly. 

In [ ]:
import plotly.express as px

def plot_3d(df, x_axis, y_axis, z_axis): 
    fig = px.scatter_3d(
        df.sort_values('label'),
        x = x_axis, 
        y = y_axis, 
        z = z_axis,
        color = 'label',
    )

    fig.show()

In [ ]:
plot_3d(df, 'pc1', 'pc2', 'pc3')

# t-SNE 

Whereas PCA is a **linear** dimensionality reduction technique, t-distributed Stochastic Neighbor Embedding (t-SNE) is a **nonlinear** dimensionality reduction technique. This means this algorithm, unlike PCA, allows us to separate data that cannot be separated by any straight line - making it more appropriate for certain datasets. 


The t-SNE algorithm models the probability distribution of neighbors around each point. Here, the term neighbors refers to the set of points which are closest to each point. In the original, high-dimensional space this is modeled as a Gaussian distribution. In the 2-dimensional output space this is modeled as a t-distribution. The goal of the procedure is to find a mapping onto the 2-dimensional space that minimizes the differences between these two distributions over all points. The fatter tails of a t-distribution compared to a Gaussian help to spread the points more evenly in the 2-dimensional space.

t-SNE was introduced in 2008 by van der Maaten and Hinton [[link to paper]](https://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf)

In [ ]:
from sklearn.manifold import TSNE

time_start = time.time()

tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_data = tsne.fit_transform(X)

time_tsne = time.time() - time_start
print(f't-SNE done in {time.time() - time_start} seconds!')

times['tsne'] = time_tsne

The computation takes some time. t-SNE is quite heavy on system resources, especially compared to PCA.

In [ ]:
df = (
    df
    .assign(tsne1_2d = tsne_data[:, 0])
    .assign(tsne2_2d = tsne_data[:, 1])
)

df.head()

In [ ]:
plot_2d(df, 'tsne1_2d', 'tsne2_2d')

It seems like the digits are very clearly clustered in their own subgroups.

In [ ]:
plot_2d(df, 'pc1', 'pc2')

### 3D Visualization
Just like with PCA, we can also use t-SNE for a 3D visualisation. 

In [ ]:
rnd = np.random.permutation(df.shape[0])

In [ ]:
size = 10000
data = X[rnd[:size]]
data.shape

As the computation would take rather a long time for many features and many samples, we will use only 10,000 samples to run the algorithm on. 

In [ ]:
time_start = time.time()

tsne = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=300)
tsne_data_3d = tsne.fit_transform(data)

print(f't-SNE done in {time.time() - time_start} seconds!')

Reducing the number of samples, even while increasing the number of components, leads to a faster result.

In [ ]:
df_subset = (
    df
    .iloc[rnd[:size]]
    .assign(tsne1_3d = tsne_data_3d[:, 0])
    .assign(tsne2_3d = tsne_data_3d[:, 1])
    .assign(tsne3_3d = tsne_data_3d[:, 2])
)

df_subset.head()

In [ ]:
plot_3d(df_subset, 'tsne1_3d', 'tsne2_3d', 'tsne3_3d')

As we have seen, t-SNE can be rather slow - a lot slower than PCA - on the high dimensional data that we have. 

The scikit-learn documentation recommends the following:
> It is highly recommended to use another dimensionality reduction method (e.g. PCA for dense data or TruncatedSVD for sparse data) to reduce the number of dimensions to a reasonable amount (e.g. 50) if the number of features is very high. This will suppress some noise and speed up the computation of pairwise distances between samples. 

### <mark>Exercise</mark>

The t-SNE algorithms has multiple parameters that can be set. One of the main parameters controlling the fitting is _perplexity_. The perplexity is related to the number of nearest neighbors considered when matching the original and fitted distributions for each point. Larger datasets usually require a larger value for perplexity, while smaller datasets suffice with focusing on the closest other points. A value between 5 and 50 is recommended, and different values can result in significantly different results. 

**Exercise:** Experiment with various values for _perplexity_ and observe how it influences your final result. For computational purposes, consider a dataset that is smaller than the original, e.g. 10.000 data samples. Feel free to set this value to whatever suits you, but keep in mind that a good value for perplexity is related to the size of the dataset. 

Also feel free to experiment with other parameters, such as _learning rate_ or _number of iterations_ (related to the optimisation process). See [scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) for more information.

Keep in mind that t-SNE is not deterministic; it does not necessarily produce similar output on successive runs with the same parameters. 

In [ ]:
help(TSNE)

In [ ]:
size = 10000
data = X[rnd[:size]]
data.shape

In [ ]:
from sklearn.manifold import TSNE

time_start = time.time()

tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_data = tsne.fit_transform(data)

time_tsne = time.time() - time_start
print(f't-SNE done in {time.time() - time_start} seconds!')

In [ ]:
exercise_df = (
    df
    .assign(tsne1_2d = tsne_data[:, 0])
    .assign(tsne2_2d = tsne_data[:, 1])
)

exercise_df.head()

In [ ]:
plot_2d(exercise_df, 'tsne1_2d', 'tsne2_2d')

### t-SNE with PCA

So let's try t-SNE with PCA! 

In [ ]:
print(f'Explained variance with 50 components: {np.sum(pca.explained_variance_ratio_[:50])}')

In [ ]:
pca_data_50 = pca_data[:, :50]
pca_data_50.shape

Ensure we have the right data. The shapes look good. 

In [ ]:
time_start = time.time()

tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_data = tsne.fit_transform(pca_data_50)

time_tsne_pca = time.time() - time_start
print(f't-SNE done in {time_tsne_pca} seconds!')

times['tsne-PCA'] = time_tsne_pca

In [ ]:
df = (
    df
    .assign(tsne1_pca = tsne_data[:, 0])
    .assign(tsne2_pca = tsne_data[:, 1])
)

df.head()

We see that the values between t-sne and t-sne with PCA differ. 

In [ ]:
plot_2d(df, 'tsne1_pca', 'tsne2_pca')

In [ ]:
plot_2d(df, 'tsne1_2d', 'tsne2_2d')

As t-SNE is an iterative approach, it cannot be applied on another dataset, unlike PCA. PCA computes a global covariance matrix and uses that to reduce the data, that matrix can be applied to a new set of data. Although the implementation in scikit-learn has a `.fit_transform()` method, it does not have a `.transform()` method to use for new data. 

Therefore, t-SNE is mostly used to visualise and understand high-dimensional data, while PCA is more often used for dimensionality reduction for clustering and/or supervised learning algorithms. 


# UMAP

Uniform Manifold Approximation and Projection (UMAP) is a dimensionality reduction technique first published in 2018. [[link to paper]](https://arxiv.org/pdf/1802.03426.pdf) UMAP at its core is very similar to t-SNE, although it aims to alleviate some of the challenges with t-SNE. 

The mathematics behind UMAP is quite advanced and relies on the following three assumptions: 
1. The data is uniformly distributed on a Riemannian manifold;
2. The Riemannian metric is locally constant (or can be approximated as such);
3. The manifold is locally connected.

UMAP essentially builds a representation of a weighted graph, where the edge weights represent the likelihood that two points are connected. This likelihood that two points are connected is determined in the following way: each point extends a radius outward, the size of which is determined by the point's distance to its _nth_ nearest neighbor. Points whose radii overlap are considered connected, and the likelihood of connection decreases as the radius grows. UMAP ensures that the local structure is perserved in balance with the global structure by stipulating that each point must be connected to at least its closest neighbor. 

This constructed graph is then projected to a lower dimensionality. The layout of the low-dimensional version of the graph is optimised in a process similar to that of t-SNE, but remarkably faster. 

A deeper dive into UMAP theory: [[link]](https://pair-code.github.io/understanding-umap/supplement.html)

In [ ]:
import umap

help(umap.UMAP)

In [ ]:
time_start = time.time()
umap_transformer = umap.UMAP(n_components=3)

umap_data = umap_transformer.fit_transform(X)

time_umap = time.time() - time_start
print(f'UMAP done in {time_umap} seconds!')
times['umap'] = time_umap

Although it seems to take longer than PCA, it UMAP is a lot quicker than t-SNE! 

In [ ]:
df = (
    df
    .assign(umap1 = umap_data[:, 0])
    .assign(umap2 = umap_data[:, 1])
    .assign(umap3 = umap_data[:, 2])
)

df.head()

We can run UMAP for three components, and use the first two to create a 2D visualisation - just like with PCA. This removes the need to  rerun UMAP to create a 3D visualization.

In [ ]:
# UMAP 2D
plot_2d(df, 'umap1', 'umap2')

In [ ]:
# UMAP 3D
plot_3d(df, 'umap1', 'umap2', 'umap3')

Whereas t-SNE has `perplexity` as the most important parameter to set, UMAP also has a few tunable parameters that determine the end result. 

* `n_neighbors`: the number of approximate nearest neighbors to construct the high-dimensional graph
* `min_dist`: the minimum distance between points in low-dimensional space. 


### <mark>Exercise</mark>
Experiment with various values for `n_neighbors` and `min_dist` and see how this influences your resulting visualisation. 

In [ ]:
help(umap.UMAP)

In [ ]:
import umap

time_start = time.time()
umap_transformer = umap.UMAP(n_components=3)

umap_data = umap_transformer.fit_transform(X)

time_umap = time.time() - time_start
print(f'UMAP done in {time_umap} seconds!')
times['umap'] = time_umap

In [ ]:
exercise_df = (
    df
    .assign(umap1 = umap_data[:, 0])
    .assign(umap2 = umap_data[:, 1])
    .assign(umap3 = umap_data[:, 2])
)

exercise_df.head()

In [ ]:
# UMAP 2D
plot_2d(exercise_df, 'umap1', 'umap2')

In [ ]:
# UMAP 3D
plot_3d(exercise_df, 'umap1', 'umap2', 'umap3')

Unlike t-SNE, UMAP can be applied to transform new data. This makes it not only useful for generating visualisations, but also gives us the option to use it in machine learning tasks. It is, however, quite a bit slower than PCA. 

# Conclusion

We have seen three different techniques for dimensionality reduction for visualisation.

**Principal Component Analysis** is a linear projection-based transformation technique that compresses a dataset onto a lower-dimensional feature subspace, while maintaining as much relevant information as possible. It ais to find the directions of maximum variance in the high-dimensional data, and projects it onto a new subspace with equal or fewer dimensions than the original. While it can be used for visualisation and exploratory data analysis or to reduce required storage space and improve computational efficiency, it is in practice also used to improve the predictive performance for both supervised methods and clustering. 

**t-Distributed Stochastic Neighbor Embedding** is a non-linear manifold learning based transformation technique. It is not able to transform new data, and not widely used to improve predictive performance for machine learning models. However, it is an incredibly powerful - though slow - technique that is especially useful for exploratory data analysis and visualisation. With higher dimensional data, it is recommended to first apply another dimensionality reduction technique such as PCA to reduce the number of dimensions to no more than 50 before t-SNE is applied.

**Uniform Manifold Approximation and Projection** is another non-linear manifold learning based transformation technique that offers a number of advantages over t-SNE, including increased speed and better preservation of the data's global structure. It is able to transform new data and can therefore be used both for visualisation as well as a dimensionality reduction technique to improve predictive performance, although PCA is more commonly used for the latter. 

### Computation time

In [ ]:
for transformer, time in times.items():
    print(f'{transformer.replace("-", " & ")} took {time} seconds.')

### Visualisations

In [ ]:
# PCA
plot_2d(df, 'pc1', 'pc2')

In [ ]:
# t-SNE
plot_2d(df, 'tsne1_2d', 'tsne2_2d')

In [ ]:
# t-SNE with PCA
plot_2d(df, 'tsne1_pca', 'tsne2_pca')

In [ ]:
# UMAP
plot_2d(df, 'umap1', 'umap2')